<a href="https://colab.research.google.com/github/GruAna/VU/blob/master/train_EasyOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EasyOCR

## Packages

In [1]:
import os
import cv2 as cv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/Colab_Notebooks/VU/utils.py /content
!mv /content/utils.py /content/utilities.py

In [4]:
from utilities import *

## Setup

## Dataset loading (training data)

**CTW1500 dataset**

Get images. Update location of images **manually**.

In [5]:
# path to image directory, get full path to all files
imgs_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/FewImages/images/'
(_, _, filenames) = next(os.walk(imgs_dir))
filenames.sort()
list_img_paths = [os.path.join(imgs_dir, file) for file in filenames]
n_imgs = len(list_img_paths)

In [6]:
# load images
train_images = [(cv.imread(file)) for file in list_img_paths]

Get paths to files with labels. Update location **manually**.

In [7]:
# path to label directory, get full path to all files
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/FewImages/labelsxml/'
(_, _, xml_files) = next(os.walk(labels_dir))
xml_files.sort()
list_xml_paths = [os.path.join(labels_dir, file) for file in xml_files]

Get ground truths from all xml files

In [9]:
ground_truth = []

for i, file in enumerate(list_xml_paths):
     ground_truth.append(read_gt_ctw_train(file))

# ground_truth is in the is a list of tuples, where first is the gt word 
# and second is an array of top left and bottom right coordinates
# format: ('text', [[tl,tl],[br,br]])

Crop images with multiple words to one image = one word, save and create corresponding label .txt file (one for all images, called `gt.txt`)

In [11]:
image_text_crop(train_images, filenames, ground_truth)

20it [00:01, 10.74it/s]


Convert gt.txt to labels.csv

In [12]:
import csv

with open('./results/gt.txt', 'r') as infile, open('./results/labels.csv', 'w') as outfile:
     stripped = (line.strip() for line in infile)
     lines = (line.split("\t") for line in stripped if line)
     writer = csv.writer(outfile)
     writer.writerow(['filename', 'words'])
     writer.writerows(lines)

Clone EasyOCR trainer

In [17]:
! git clone https://github.com/JaidedAI/EasyOCR

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2237, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 2237 (delta 0), reused 1 (delta 0), pack-reused 2235
Receiving objects: 100% (2237/2237), 148.54 MiB | 15.18 MiB/s, done.
Resolving deltas: 100% (1364/1364), done.
Checking out files: 100% (255/255), done.


In [18]:
! mkdir ./EasyOCR/trainer/all_data/en_val
! mkdir ./EasyOCR/trainer/all_data/en_train_filtered

In [25]:
! cp /content/results/labels.csv /content/EasyOCR/trainer/all_data/en_val
! cp /content/results/labels.csv /content/EasyOCR/trainer/all_data/en_train_filtered
! cp /content/results/*.tif /content/EasyOCR/trainer/all_data/en_train_filtered
! cp /content/results/*.tif /content/EasyOCR/trainer/all_data/en_val

In [20]:
% cd ./EasyOCR/trainer

/content/EasyOCR/trainer


## Training
### Training setup

In [21]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: en_train_filtered
all_data/en_train_filtered
sub-directory:	/en_train_filtered	 num samples: 178
num total samples of en_train_filtered: 178 x 1.0 (total_data_usage_ratio) = 178
num samples of en_train_filtered per batch: 32 x 1.0 (batch_ratio) = 32


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 178
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 600 20 1 256 256 97 34 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Con